Initialize Sentiment Analyzer

In [ ]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

Updating Vader Lexicon with External Lexicon

In [ ]:
# import csv
# import pandas as pd

# # stock market lexicon
# stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
# stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
# stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
# stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}
# stock_lex_scaled = {}
# for k, v in stock_lex.items():
#     if v > 0:
#         stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
#     else:
#         stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# # # Loughran and McDonald
# # positive = []
# # with open('lexicon_data/lm_positive.csv', 'r') as f:
# #     reader = csv.reader(f)
# #     for row in reader:
# #         positive.append(row[0].strip())
    
# # negative = []
# # with open('lexicon_data/lm_negative.csv', 'r') as f:
# #     reader = csv.reader(f)
# #     for row in reader:
# #         entry = row[0].strip().split(" ")
# #         if len(entry) > 1:
# #             negative.extend(entry)
# #         else:
# #             negative.append(entry[0])

# final_lex = {}
# # final_lex.update({word:2.0 for word in positive})
# # final_lex.update({word:-2.0 for word in negative})
# final_lex.update(stock_lex_scaled)
# final_lex.update(sia.lexicon)
# sia.lexicon = final_lex

Scraping FB posts from Business Times

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint
import pandas as pd

date_sentiments = {}

SP500=pd.read_csv('/Users/zhoujiawang/Desktop/constituents_csv.csv')

for i in SP500['Name']:
    page = urlopen('https://www.businesstimes.com.sg/search/'+i+'?page=1').read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})    
    try:
        post=posts[0]
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(i, []).append(sentiment)
        print(date_sentiment)
    except Exception:pass

print(date_sentiment)


78

In [21]:
page = urlopen('https://www.businesstimes.com.sg/search/facebook?page=1').read()
soup = BeautifulSoup(page, features="html.parser")
posts = soup.findAll("div", {"class": "media-body"})
for post in posts:
    url = post.a['href']
url
posts[0]

<div class="media-body">
<h4><a href="https://www.businesstimes.com.sg/technology/facebook-says-ai-getting-better-at-spotting-unwanted-content">Facebook says AI getting better at spotting unwanted content</a></h4><time>15 Nov 2019</time><p>FACEBOOK on Wednesday said its software is getting more skilled at spying banned content on the social network and then working with humans to quickly remove terrorist videos and more.
</p></div>

Algorithm

In [5]:
from __future__ import (absolute_import, division, print_function,
                            unicode_literals)

# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')

import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)
        

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2018,11,25),
        reverse = False
    )
    
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()


Starting Portfolio Value: 100000.00
2018-07-26, Close, 176.26
Sentiment Score: 0.593
2018-07-27, Close, 174.89
Sentiment Score: 0.603
2018-07-30, Close, 171.06
Sentiment Score: 0.603
2018-07-31, Close, 172.58
Sentiment Score: 0.973
2018-08-01, Close, 171.65
Sentiment Score: 0.973
2018-08-02, Close, 176.37
Sentiment Score: 0.991
2018-08-03, Close, 177.78
Sentiment Score: 0.991
2018-08-06, Close, 185.69
Sentiment Score: 0.991
2018-08-07, Close, 183.81
Sentiment Score: 0.991
2018-08-08, Close, 185.18
Sentiment Score: 0.996
2018-08-09, Close, 183.09
Sentiment Score: 0.996
2018-08-10, Close, 180.26
Sentiment Score: 0.996
2018-08-13, Close, 180.05
Sentiment Score: 0.996
2018-08-14, Close, 181.11
Sentiment Score: 0.88
2018-08-15, Close, 179.53
Sentiment Score: 0.88
2018-08-16, Close, 174.70
Sentiment Score: 0.88
2018-08-17, Close, 173.80
Sentiment Score: 0.88
2018-08-20, Close, 172.50
Sentiment Score: 0.88
2018-08-21, Close, 172.62
Sentiment Score: 0.88
2018-08-22, Close, 173.64
Sentiment Sco